# Inteligência Artificial Aplicada a Finanças

# Projeto 3 - Otimização Avançada de Portfólio com IA

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.6


![title](imagens/projeto3.png)

## Parte 1 - Coleta Automática dos Dados

Neste projeto vamos construir um sistema de IA para otimização de portfólio de investimentos, que será capaz de aprender de forma automática como otimizar o lucro ao longo do tempo com diferentes combinações de ativos financeiros, especificamente ações listadas na NASDAQ.

### Instalando e Carregando Pacotes

In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [3]:
!pip install -q -U pandas_datareader

In [4]:
!pip install -q -U holidays

In [5]:
# Imports
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import time
import pandas as pd
import numpy as np
import requests
import holidays
import bs4
import pandas_datareader
from random import randint
from datetime import date
from bs4 import BeautifulSoup
from pandas_datareader import data as pdr

In [6]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

numpy             1.19.2
pandas            1.1.3
bs4               4.8.2
holidays          0.10.3
requests          2.22.0
pandas_datareader 0.9.0
Data Science Academy


### Função Para Obter Dados de Ações do Yahoo Finance

In [7]:
def retorna_acoes(ticker_list, start, end):
    list_of_df = []
    not_found = []
    
    for ticker in ticker_list:
        try:
            df = pdr.get_data_yahoo(ticker, start, end)
            df = df.reset_index()
            df.rename(columns = {'Date':'date'})
            df['ticker'] = [ticker] * len(df)
            list_of_df.append(df)
        except ValueError:
            print("Não Encontrado: " + ticker)
            not_found.append(ticker)
    
    return list_of_df, not_found

### Função Para Scraping de Dados Históricos da NASDAQ

In [8]:
def scrape_nasdaq(ticker_list):
    tckr_data = {}
    count = 1
    
    for ticker in ticker_list:
        print("Scraping count: " + str(count))
        
        url = 'https://old.nasdaq.com/symbol/' + ticker + '/historical'
        driver.get(url)
        
        # Coleta de 10 anos de dados
        data_range = driver.find_elements_by_name('ddlTimeFrame')
        
        if len(data_range) > 0:
            for option in data_range[0].find_elements_by_tag_name('option'):
                if option.text == '10 Years':
                    option.click()
                    break
            time.sleep(5)
            page_source = driver.page_source
    
            # Gerando objeto soup para o parse de dados da URL
            soup = BeautifulSoup(page_source, 'lxml')
    
            # Definindo a regra de pesquisa na tabela de dados históricos
            tags = soup.find_all('div', id = "historicalContainer")
    
            # Iniciando a busca
            temp_data = []
    
            for tag in tags:
                rows = tag.findAll('tr')
                for tr in rows:
                    cols = tr.findAll('td')
                    val = [tr.text for tr in cols]
                    temp_data.append(val)
            
            # Limpando os dados
            for i in range(len(temp_data)):
                to_process = temp_data[i]
    
                for i in range(len(to_process)):
                    temp = to_process[i]
                    temp = temp.strip()
                    to_process[i] = temp
         
            # Obtendo os dados
            temp_data = temp_data[2:]
            
            if tckr_data.get(ticker) == None:
                tckr_data[ticker] = temp_data
        
            # Pausa na coleta para evitar bloqueio no site da Nasdaq
            print("Random Sleep")
            sleep(randint(2, 4))
            count += 1
        
        else:
            print("Não Encontrado" + ticker)
        
    # Preparando as listas para o dataframe
    date = []
    Open = []
    high = []
    low = []
    close = []
    volume = []
    ticker = []

    # Coletando e classificando os dados
    for key, value in tckr_data.items():
        for data in value:
            date.append(data[0].replace('/','-'))
            Open.append(data[1])
            high.append(data[2])
            low.append(data[3])
            close.append(data[4])
            volume.append(data[5].replace(',',''))
            ticker.append(key)  
            
    # Dicionário final com os dados
    final_dict = {'date' : date, 'Open':Open, 'High':high, 'Low':low, 'Close':close, 'Volume':volume, 'ticker':ticker}
    
    # Gerando o dataframe a partir do dicionário
    df = pd.DataFrame(final_dict)
    cols = ['date', 'Open', 'High', 'Low', 'Close', 'Volume', 'ticker']
    df = df[cols]

    return df

### Function Para Obter as Cotações Históricas Usando as Duas Funções Acima

In [9]:
def get_stocks_data(ticker_set, start, end):
    
    # Retorna ações que que serão coletadas
    stocks_1, not_found = retorna_acoes(ticker_set, start, end)
    
    if len(not_found) > 0:
    
        # Dados das ações da Nasdaq
        stocks = scrape_nasdaq(not_found)
    
        # Merging em um dataframe
        stocks_1 = pd.concat(stocks_1)

        stocks_1 = stocks_1.rename(columns = {'Date' : 'date'})
        stocks_1 = stocks_1[['date', 'Open', 'High', 'Low', 'Close', 'Volume', 'ticker']]
        stocks_1['date'] = stocks_1['date'].dt.date

        stocks = stocks.rename(columns = {'Ticker' : 'ticker'})
        stocks[['Open', 'High', 'Low', 'Close', 'Volume']] = stocks[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
    
        stocks_final = pd.concat([stocks_1, stocks])
        return stocks_final
    
    else:
        return pd.concat(stocks_1)

In [10]:
# 16 Ações que serão coletadas
tickers = ['AAPL', 'XOM' ,'VMC', 'BA', 'AMZN', 'TGT', 'WMT', 'KO', 'UNH', 'JPM', 'GOOGL', 'STT', 'MSFT', 'VZ', 'XEL', 'SPG']

In [11]:
# Coletando as cotações históricas de 10 anos
df = get_stocks_data(tickers, start = '2009-01-01', end = '2019-12-31')

In [12]:
df.sample(5)

,Date,High,Low,Open,Close,Volume,Adj Close,ticker
148,2009-08-05,85.750000,83.019997,85.639999,84.290001,7279400.0,84.290001,AMZN
2180,2017-08-30,79.099998,78.519997,78.820000,78.540001,5829800.0,73.457710,WMT
705,2011-10-19,79.239998,77.919998,78.570000,78.430000,20910900.0,56.269211,XOM
2409,2018-07-30,53.000000,52.080002,52.270000,52.490002,16725000.0,47.719433,VZ
1144,2013-07-22,56.669998,55.950001,56.169998,56.560001,15999700.0,45.984550,JPM


In [13]:
# Ajustando o dataframe apenas com as colunas que nos interessam
df = df[['Date', 'Open', 'High', 'Low', 'Close', 'ticker']]

In [14]:
# Amostra do que foi coletado
df.sample(5)

,Date,Open,High,Low,Close,ticker
220,2009-11-16,7.338572,7.428571,7.321786,7.379643,AAPL
2052,2017-02-28,165.470001,166.759995,164.899994,165.380005,UNH
556,2011-03-18,81.769997,82.330002,80.849998,80.849998,XOM
918,2012-08-23,30.389999,30.400000,30.080000,30.260000,MSFT
1858,2016-05-20,67.389999,69.139999,67.000000,68.660004,TGT


In [15]:
# Salvando em formato csv
df.to_csv('dados/stocks.csv')

# Fim